In [1]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
import matplotlib.pyplot as plt
torch.manual_seed(0) # Set for testing purposes, please do not change!

In [612]:
class Autoencoder(nn.Module):
    def __init__(self, char_num_embeddings=10, char_embedding_dim=5, word_embedding_dim=2):
        super(Autoencoder, self).__init__()
        self.char_embedding_dim = char_embedding_dim
        self.embedding = nn.Sequential(
            nn.Embedding(char_num_embeddings, char_embedding_dim, max_norm=True)
        )
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=(3, char_embedding_dim), padding=(2, 0)),
            nn.ReLU(True),
            nn.Conv2d(8, 16, kernel_size=(3, 1), padding=(2, 0)),
            nn.Flatten(1, 3),
            nn.Linear(9 * 16, word_embedding_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(word_embedding_dim, 9 * 16),
            nn.Unflatten(1, (16, 9, 1)),
            nn.ConvTranspose2d(16, 9, kernel_size=(1, 1)),
            nn.ReLU(True),
            nn.ConvTranspose2d(9, 1, kernel_size=(1, 5), padding=(2, 0)),
        )
    def forward(self, x):
        x = x.view(len(x), 1, self.char_embedding_dim)
        return self.decoder(self.encoder(self.embedding(x)))

In [622]:
device = 'cpu'
lr = 0.00001

gen = Autoencoder(char_num_embeddings=10, char_embedding_dim=5, word_embedding_dim=2).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)

In [623]:
gen(torch.LongTensor([[1, 2, 4, 5, 9], [1, 2, 4, 5, 9]]))

tensor([[[[ 0.2607,  0.4310,  0.1563,  0.4537,  0.2289],
          [ 0.4093,  0.5386,  0.3286,  0.2687,  0.2738],
          [ 0.0310,  0.5530,  0.2436,  0.7137,  0.2564],
          [ 0.4468,  0.2088,  0.1503,  0.2652,  0.1198],
          [ 0.5723,  0.2027, -0.0158, -0.0767, -0.1016]]],


        [[[ 0.2607,  0.4310,  0.1563,  0.4537,  0.2289],
          [ 0.4093,  0.5386,  0.3286,  0.2687,  0.2738],
          [ 0.0310,  0.5530,  0.2436,  0.7137,  0.2564],
          [ 0.4468,  0.2088,  0.1503,  0.2652,  0.1198],
          [ 0.5723,  0.2027, -0.0158, -0.0767, -0.1016]]]],
       grad_fn=<ConvolutionBackward0>)

In [617]:
gen.decoder(torch.FloatTensor([[ 0.0708, -0.1364], [ 0.0708, -0.1364]]))

tensor([[[[-0.3032,  0.2617,  0.0890,  0.4227, -0.3036],
          [-0.0009,  0.0047,  0.1346, -0.0153,  0.3670],
          [-0.0304, -0.1539, -0.0055,  0.3631, -0.4502],
          [-0.0032, -0.2092,  0.3232, -0.2495,  0.3346],
          [ 0.0921,  0.0023,  0.2342, -0.2408,  0.2772]]],


        [[[-0.3032,  0.2617,  0.0890,  0.4227, -0.3036],
          [-0.0009,  0.0047,  0.1346, -0.0153,  0.3670],
          [-0.0304, -0.1539, -0.0055,  0.3631, -0.4502],
          [-0.0032, -0.2092,  0.3232, -0.2495,  0.3346],
          [ 0.0921,  0.0023,  0.2342, -0.2408,  0.2772]]]],
       grad_fn=<ConvolutionBackward0>)